In [4]:
import numpy as np
import math
from scipy.integrate import quad
from scipy.special import erf

def integral_by_quad(lam_s, lam_t, lam_u, beta, tau):
    """
    주어진 구간 [lam_s, lam_t]에서
    exp(lam - beta^2 * (lam - lam_u)**2)를
    수치적으로 적분하여 결과와 추정 오차를 반환.
    """

    def integrand(lam, lam_u, beta, tau):
      """
      적분에 쓰일 integrand 함수.
      exp(lam - beta^2 * (lam - lam_u)^2)를 반환.
      """
      return np.exp(tau*lam - beta**2 * (lam - lam_u)**2)

    val, err = quad(integrand, lam_s, lam_t, args=(lam_u, beta, tau))
    return val

def integral_by_closed(lam_s, lam_t, lam_u, beta, tau):
    """
    [lam_s, lam_t] 구간에서
    exp(lam - beta^2*(lam - lam_u)^2) 적분의 닫힌형 해를 반환한다.
    """
    prefactor = np.exp(tau*lam_u + tau**2/(4*beta**2)) * np.sqrt(np.pi)/(2*beta)
    upper = beta*(lam_t-lam_u-(tau/(2*beta**2)))
    lower = beta*(lam_s-lam_u-(tau/(2*beta**2)))
    return prefactor * (erf(upper) - erf(lower))

In [7]:
import matplotlib.pyplot as plt

# log_scales : -1 ~ 1 구간 10등분, Goal : -6 ~ 6 구간까지 증가시켜야함
log_scales = np.linspace(-3, 3, 10)
scales = np.exp(log_scales)

# 하이퍼파라미터 설정
lam_min = -5.0778
lam_max = 5.7618
lam_range = lam_max - lam_min

for NFE in range(5, 11):
    allclose_list = []

    for step in range(NFE):
        h = lam_range / NFE
        betas = 1/(scales*h)

        for tau in np.linspace(0, 1, 2):

            for r_u in [1, 0, -1, -2]:
                # 적분 구간 [lam_s, lam_t] 정의
                lam_s = lam_min + step * h
                lam_t = lam_min + (step+1) * h
                lam_u = lam_s + r_u * h

                # 각각 수치적분과 닫힌형 해로 적분
                inte_by_quad = np.array([integral_by_quad(lam_s, lam_t, lam_u, beta, tau) for beta in betas])
                inte_by_closed = np.array([integral_by_closed(lam_s, lam_t, lam_u, beta, tau) for beta in betas])

                # 두 방식의 적분 결과를 비교
                allclose = np.allclose(inte_by_quad, inte_by_closed)
                allclose_list.append(allclose)

    allclose_list = np.array(allclose_list)
    print('NFE :', NFE, ' result :', 'Pass' if np.prod(allclose_list) else 'Fail')

NFE : 5  result : Fail
NFE : 6  result : Fail
NFE : 7  result : Fail
NFE : 8  result : Fail
NFE : 9  result : Fail
NFE : 10  result : Fail
